In [22]:
import pandas as pd
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf

In [33]:
#Read in the data
dfBase = pd.read_csv("Data/OffensiveStats2017.csv")
dfBase = dfBase.dropna()

#Get rid of players with AB < 50
dfBase = dfBase.drop(dfBase[dfBase['AB'] < 50].index)

#Drop some stuff that was type object from original sheet
dfBase = dfBase.drop('SO%', 1)
dfBase = dfBase.drop('BB%', 1)
dfBase = dfBase.drop('SB%', 1)

#Calculate percentages of certain occurances
dfBase['SOper'] = dfBase['SO']/dfBase['AB']
dfBase['BBper'] = dfBase['BB']/dfBase['AB']
dfBase['HRper'] = dfBase['HR']/dfBase['AB']
dfBase['SBper'] = dfBase['SB']/(dfBase['SB']+dfBase['CS'])
dfBase['RBIperAB'] = dfBase['RBI']/dfBase['AB']

#Drop uneeded rows
dfBase = dfBase.drop('SO', 1)
dfBase = dfBase.drop('BB', 1)
dfBase = dfBase.drop('HR', 1)
dfBase = dfBase.drop('H', 1)
dfBase = dfBase.drop('2B', 1)
dfBase = dfBase.drop('R', 1)
dfBase = dfBase.drop('RBI', 1)
dfBase = dfBase.drop('SB', 1)
dfBase = dfBase.drop('CS', 1)
dfBase = dfBase.drop('3B', 1)

dfBase = dfBase.rename(columns={'OPS+':'OPSplus'})


dfBase.info()
#dfBase.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 593 entries, 1 to 1625
Data columns (total 14 columns):
G           593 non-null int64
PA          593 non-null int64
AB          593 non-null float64
BA          593 non-null float64
OBP         593 non-null float64
SLG         593 non-null float64
OPS         593 non-null float64
OPSplus     593 non-null float64
TB          593 non-null float64
SOper       593 non-null float64
BBper       593 non-null float64
HRper       593 non-null float64
SBper       471 non-null float64
RBIperAB    593 non-null float64
dtypes: float64(12), int64(2)
memory usage: 69.5 KB


In [28]:
def forward_select(df, resp_str , maxk):
    
    remaining = set(df.columns)
    remaining.remove(resp_str)
    selected = []
    numselected = 1
    score_crnt, score_new = 0.0, 0.0
    while remaining and score_crnt == score_new:
        score_array = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(resp_str,' + '.join(selected + [candidate]))
            score = smf.ols(formula, df).fit().rsquared_adj
            score_array.append((score, candidate))
        score_array.sort()
        score_new, best_option = score_array.pop()
        if score_crnt < score_new and numselected <= maxk:
            remaining.remove(best_option)
            selected.append(best_option)
            score_crnt = score_new
            numselected += 1
    formula = "{} ~ {} + 1".format(resp_str,' + '.join(selected))
    model = smf.ols(formula, df).fit()
    return model

In [32]:
model = forward_select(dfBaseballO, "OPSplus", 5)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                OPSplus   R-squared:                       0.986
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     8116.
Date:                Sat, 23 Dec 2017   Prob (F-statistic):               0.00
Time:                        17:50:33   Log-Likelihood:                -1734.5
No. Observations:                 593   AIC:                             3481.
Df Residuals:                     587   BIC:                             3507.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept    -94.3080      1.702    -55.415      0.000       -97.650   -90.966
OPS          225.8375      6.130     36.842      0.000       213.798   237.877
OBP           73.5042     11.631      6.320      0.000        50.660    96.348
RBIperAB     -19.1712      7.118     -2.693      0.007       -33.151    -5.191
HRper         64.1151     21.061      3.044      0.002        22.751   105.479
SOper         -7.8037      2.622     -2.976      0.003       -12.954    -2.653
==============================================================================
Omnibus:                       39.498   Durbin-Watson:                   1.907
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               78.368
Skew:                          -0.412   Prob(JB):                     9.61e-18
Kurtosis:                       4.579   Cond. No.                         161.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [36]:
dfBaseballO = dfBaseballO.drop('OBP', 1)
dfBaseballO = dfBaseballO.drop('SLG', 1)
dfBaseballO = dfBaseballO.drop('OPS', 1)
dfBaseballO = dfBaseballO.drop('RBIperAB', 1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 593 entries, 1 to 1625
Data columns (total 10 columns):
G          593 non-null int64
PA         593 non-null int64
AB         593 non-null float64
BA         593 non-null float64
OPSplus    593 non-null float64
TB         593 non-null float64
SOper      593 non-null float64
BBper      593 non-null float64
HRper      593 non-null float64
SBper      471 non-null float64
dtypes: float64(8), int64(2)
memory usage: 51.0 KB


In [37]:
model = forward_select(dfBaseballO, "OPSplus", 5)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                OPSplus   R-squared:                       0.971
Model:                            OLS   Adj. R-squared:                  0.971
Method:                 Least Squares   F-statistic:                     3925.
Date:                Sat, 23 Dec 2017   Prob (F-statistic):               0.00
Time:                        18:16:32   Log-Likelihood:                -1945.5
No. Observations:                 593   AIC:                             3903.
Df Residuals:                     587   BIC:                             3929.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept    -85.7477      2.431    -35.272      0.000       -90.522   -80.973
BA           548.5119      7.888     69.538      0.000       533.020   564.004
HRper        684.8383     15.222     44.989      0.000       654.941   714.735
BBper        191.5535      6.386     29.995      0.000       179.011   204.096
SOper         -7.8070      3.943     -1.980      0.048       -15.551    -0.063
AB             0.0025      0.002      1.358      0.175        -0.001     0.006
==============================================================================
Omnibus:                        3.157   Durbin-Watson:                   2.064
Prob(Omnibus):                  0.206   Jarque-Bera (JB):                3.521
Skew:                          -0.031   Prob(JB):                        0.172
Kurtosis:                       3.372   Cond. No.                     2.01e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.01e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""